In [ ]:
spark.stop()

In [ ]:
# Convert Json to CSV

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window
import numpy as np
import pandas as pd
from time import time
from timeit import default_timer as timer
import random
import os.path
import pickle
import pandas

dir_path = "/mnt/nvme2/chendi/BlueWhale/x-deeplearning/xdl-algorithm-solution/DIEN/data/"
local_prefix = "file://" + dir_path

t0 = timer()
spark = SparkSession\
    .builder\
    .master('yarn')\
    .appName("DIEN_DATA_Comparor") \
    .getOrCreate()

# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

def load_csv(file_path, in_type = "python"):
    label_field = StructField('label', IntegerType())
    uid_field = StructField('uid', StringType())
    asin_field = StructField('asin', StringType())
    cat_field = StructField('category', StringType())
    asin_list_field = StructField('asin_list', StringType())
    cat_list_field = StructField('cast_list', StringType())
    id_field = StructField('id', IntegerType())
    
    schema_1 = StructType([label_field, uid_field, asin_field, cat_field, asin_list_field, cat_list_field, id_field])
    schema_2 = StructType([label_field, uid_field, asin_field, cat_field, asin_list_field, cat_list_field])
    
    if in_type == "python":
        test = spark.read.schema(schema_2).option('sep', '\t').csv(file_path + "/local_test_splitByUser") 
        train = spark.read.schema(schema_2).option('sep', '\t').csv(file_path + "/local_train_splitByUser")
        return test, train
    else: 
        test = spark.read.schema(schema_1).option('sep', '\t').csv(file_path + "/local_test_splitByUser") 
        train = spark.read.schema(schema_1).option('sep', '\t').csv(file_path + "/local_train_splitByUser")
        return test, train

python_test, python_train = load_csv(local_prefix + "python_output")
pyspark_test, pyspark_train = load_csv(local_prefix + "pyspark_output", in_type = "pyspark")

python_output = python_test.union(python_train)
pyspark_output = pyspark_test.union(pyspark_train)

out = python_output\
        .join(pyspark_output, [python_output.uid == pyspark_output.uid, python_output.label == pyspark_output.label], 'left_outer')\
        .filter(python_output.asin != pyspark_output.asin)\
        .filter(python_output.label != 0)

out.show(n=20, truncate=False, vertical=True)
print(out.count())





In [ ]:
aggregate_records = spark.read.parquet(local_prefix + "/pyspark_output/aggregated_records")
aggregate_records.show()

In [ ]:
spark.stop()